# Задание 1

Нужно решить задачу прогнозирования спроса

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

Посмотрим на данные.

In [2]:
train = pd.DataFrame.from_csv('train.tsv', header=0)
train.head()

,y,year,week,shift,item_id,f1,f2,f3,f4,f5,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
Num,,,,,,,,,,,,,,,,,,,,,
0,123438,2012,52,1,20442076,4915.0,38056.0,40185.0,45733.0,59710.0,...,39423.0,41765.0,52590.0,31452.0,44420.0,41865.0,52705.0,36102.0,44163.0,45239.0
1,58410,2012,52,1,20441997,2230.0,18817.0,20110.0,26368.0,31412.0,...,22830.0,25230.0,27850.0,21390.0,27090.0,23170.0,29705.0,19140.0,22055.0,23200.0
2,163930,2012,52,1,20441990,5695.0,47480.0,47619.0,89708.0,166338.0,...,14930.0,44290.0,46412.0,29320.0,21140.0,28406.0,65056.0,31886.0,48750.0,36520.0
3,53902,2012,52,1,20441989,1995.0,17146.0,20066.0,27070.0,35337.0,...,15120.0,12480.0,19780.0,7990.0,8230.0,10650.0,21920.0,13040.0,9780.0,9630.0
4,105970,2012,52,1,20441988,6515.0,49262.0,50045.0,95167.0,102390.0,...,18872.0,19328.0,37168.0,13570.0,19760.0,20208.0,34745.0,18442.0,24700.0,21793.0


In [3]:
train.shape

(72457, 65)

$\textbf{Num}$ - номер строки в выборке

$\textbf{y}$ - продажи товара c идентификатором $\textbf{item_id}$ за неделю номер $\textbf{week}$ в году $\textbf{year}$

$\textbf{shift}$ - на сколько недель значения признаков отстают от недели, записанной в week (отставание признаков нужно чтобы научиться прогнозировать продажи на 2 или 3 недели вперед)

$\textbf{f1, ..., f60}$ - признаки, посчитанные как средние продажи данного товара за разные периоды времени и по разным срезам

In [4]:
test = pd.DataFrame.from_csv('test.tsv', header=0)
test.head()

,year,week,shift,item_id,f1,f2,f3,f4,f5,f6,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
Num,,,,,,,,,,,,,,,,,,,,,
348622,2015,3,3,20447918,960.0,820.0,1128.0,1801.0,1045.0,1190.0,...,1510.0,580.0,969.0,1635.0,895.0,2140.0,1182.0,1020.0,1293.0,1290.0
348623,2015,3,3,20447902,9086.0,12585.0,11595.0,9685.0,12917.0,11157.0,...,22055.0,14235.0,21195.0,18280.0,18270.0,15851.0,16920.0,18320.0,24116.0,21307.0
348624,2015,3,3,20447732,115087.0,147287.0,176065.0,143105.0,202069.0,216920.0,...,302165.0,162232.0,221622.0,256605.0,240047.0,236630.0,206697.0,245652.0,286179.0,285904.0
348625,2015,3,3,20443951,20900.0,24420.0,27068.0,20460.0,25580.0,27135.0,...,39055.0,14445.0,22450.0,22093.0,31175.0,23355.0,15358.0,18930.0,29643.0,33970.0
348626,2015,3,3,20443944,4430.0,5864.0,3310.0,1853.0,2836.0,2807.0,...,120.0,130.0,60.0,30.0,50.0,20.0,20.0,30.0,0.0,0.0


### Предобработка признаков

Для начала посмотрим на корреляцию признаков f1-f60.

In [5]:
F = train.drop(['y', 'year', 'week', 'shift', 'item_id'], axis=1)
F.corr().round(2)

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
f1,1.00,0.91,0.89,0.89,0.93,0.93,0.89,0.88,0.92,0.94,...,0.88,0.90,0.92,0.89,0.87,0.89,0.92,0.89,0.87,0.87
f2,0.91,1.00,0.92,0.90,0.90,0.95,0.94,0.90,0.89,0.92,...,0.89,0.88,0.91,0.93,0.89,0.88,0.89,0.94,0.89,0.87
f3,0.89,0.92,1.00,0.92,0.90,0.91,0.95,0.94,0.90,0.89,...,0.92,0.88,0.88,0.91,0.93,0.89,0.88,0.89,0.94,0.89
f4,0.89,0.90,0.92,1.00,0.92,0.90,0.91,0.95,0.94,0.90,...,0.93,0.92,0.88,0.88,0.91,0.93,0.89,0.88,0.89,0.93
f5,0.93,0.90,0.90,0.92,1.00,0.92,0.90,0.90,0.95,0.94,...,0.89,0.93,0.92,0.88,0.89,0.91,0.93,0.88,0.88,0.89
f6,0.93,0.95,0.91,0.90,0.92,1.00,0.92,0.90,0.91,0.96,...,0.89,0.90,0.94,0.92,0.89,0.89,0.91,0.93,0.89,0.88
f7,0.89,0.94,0.95,0.91,0.90,0.92,1.00,0.92,0.90,0.91,...,0.90,0.89,0.89,0.94,0.92,0.89,0.89,0.91,0.93,0.89
f8,0.88,0.90,0.94,0.95,0.90,0.90,0.92,1.00,0.92,0.90,...,0.95,0.90,0.89,0.89,0.94,0.92,0.89,0.89,0.91,0.93
f9,0.92,0.89,0.90,0.94,0.95,0.91,0.90,0.92,1.00,0.92,...,0.91,0.95,0.90,0.89,0.90,0.94,0.92,0.89,0.89,0.91
f10,0.94,0.92,0.89,0.90,0.94,0.96,0.91,0.90,0.92,1.00,...,0.90,0.91,0.95,0.90,0.89,0.89,0.94,0.92,0.89,0.89


Видим, что у всех векторов признаков достаточно большая корреляция. Однако, если посмотреть внимательно, то можно заметить, что у векторов f1 и f31 корреляция вообще равна 1. То же самое с векторами f2 и f32, f3 и f33 и т. д. Проверим действительно ли f1,...,f30 = f31, ..., f60.

In [6]:
all((train[f'f{i}'] == train[f'f{i + 30}']).all() for i in range(1, 31))

True

Так и есть, уберем лишние признаки.

In [7]:
train = train.drop([f'f{i}' for i in range(31, 61)], axis=1)
test = test.drop([f'f{i}' for i in range(31, 61)], axis=1)

### Выбор модели

Подготовим метрику, которой будем оценивать решение во время кросс-валидации.

In [8]:
from sklearn.metrics import make_scorer

def smape(actual, forecast):
    assert len(actual) == len(forecast)
    n = len(actual)
    return 100 / n * np.sum(2 * np.abs(forecast - actual) / (np.abs(actual) + np.abs(forecast)))

smape_scorer = make_scorer(smape)

Будем также учитывать, что в данных у нас есть зависимость от времени.

In [9]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)

Попробуем разные модели и посмотрим значения кросс-валидации. Начнем с регрессора из xgboost.

In [12]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

X = train.drop('y', axis=1)
y = train['y']

est = XGBRegressor(n_estimators=100, max_depth=5)

scores = cross_val_score(est, X, y, cv=tscv, scoring=smape_scorer)
scores, scores.mean()

(array([ 45.97279862,  65.68913996,  40.31030494,  37.84954955,  61.25408262]),
 50.215175137996553)

Градиентный бустинг дает SMAPE около 50%. Возможно, так плохо из-за большого количества признаков или неподходящих параметров модели. Попробуем теперь случайный лес.

In [13]:
from sklearn.ensemble import RandomForestRegressor

est = RandomForestRegressor(n_estimators=100, n_jobs=-1)

scores = cross_val_score(est, X, y, cv=tscv, scoring=smape_scorer)
scores, scores.mean()

(array([ 30.21671754,  33.07195465,  23.10848769,  21.23197197,  35.25243296]),
 28.576312964112276)

Как видим, случайный лес дает результат намного лучше. Даже, используя такую простую модель, получаем SMAPE 24.3% на Private Leaderboard. Попробуем подобрать параметры модели получше.

In [64]:
from sklearn.model_selection import GridSearchCV

params = {'max_features' : range(1, 35, 2)} 

neg_smape_scorer = make_scorer(lambda a, f : -smape(a, f))
gscv = GridSearchCV(estimator=RandomForestRegressor(n_estimators=100, n_jobs=-1),
                    param_grid=params, scoring=neg_smape_scorer, cv=tscv)
gscv.fit(X, y)
         
gscv.best_params_, gscv.best_score_

({'max_features': 21}, -28.407987871804135)

Количество деревье подбирать не будем, так как случайный лес не склонен к переобучению, и можно просто взять побольше. Глубину деревьев также можно не ограничивать, так как отдельные переобученные деревья менее скоррелированны и в среднем работают лучше.

In [29]:
est = RandomForestRegressor(n_estimators=100, **gscv.best_params_)
est.fit(X, y)

submission = pd.read_csv('sample_submission.tsv')
submission['y'] = est.predict(test)
submission.to_csv('submission.tsv', sep=',', index=False)

Получили SMAPE 22.63% на Private Leaderboard.